### CLASSIFYING ENZYMES TO FUNCTIONAL CLASSES

In [1]:
# IMPORTS - PART 1
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

 * AAC - amino acid composition (frequency of each AA in a sequence) --> N x 20
 * CTD - conjoined triad descriptors () --> N x 343
 * NGRAM

In [ ]:
# READ DATA FOR SVM, kNN and RF
aac = pd.read_csv('../dataset/aac.csv', index_col=0)
ctd = pd.read_csv('../dataset/ctd.csv', index_col=0)
data = pd.read_csv('../dataset/ngram.csv', index_col=0)
data = data.join(ctd[ctd.columns[0:-1]])
data = data.join(aac[aac.columns[0:-1]])
y = data['class']
data = data[data.columns[0:-1]]

In [3]:
# NORMALIZE DATA
mean = data.mean()
std = data.std()
data = (data-mean)/std

#### Majority Classifier

In [4]:
# DEFINE MAJORITY CLASSIFIER
class Majority_class:
    def __init__(self):
        self.major = None

    def fit(self, y):
        self.major = y.mode()
        return self
        
    def predict(self, x):
        if self.major is not None:
            return np.full(len(x), self.major)
        print('Train classifier first')

In [46]:
# CROSS VALIDATE MAJORITY CLASSIFIER
results_major = []
i = 0
j = 2400
while j <= y.size:
    test_idx = np.zeros(y.size, dtype=bool)
    test_idx[i:j] = True
    test = y[test_idx]
    train = y[np.invert(test_idx)]
    major = Majority_class()
    major = major.fit(train)
    results_major.append(np.mean(major.predict(test) == y[test_idx].to_numpy()))
    i = j
    j += 2400
    if j < y.size and y.size - j < 245:
        j = y.size

In [47]:
acc_major = np.round(np.mean(results_major) * 100, decimals=3)
err_major = np.round(np.std(results_major) * 100, decimals=3)
print("Majority classifier has", acc_major, "% accuracy with", err_major, "% error\n(estimated with", len(results_major), "fold cross validation)")

Majority classifier has 33.9 % accuracy with 3.674 % error
(estimated with 10 fold cross validation)


#### Support Vector Classifier

In [48]:
clf_svm = svm.SVC()
results_svm = cross_val_score(clf_svm, data, y, cv=len(results_major))

In [54]:
acc_svm = np.round(np.mean(results_svm) * 100, decimals=3)
err_svm = np.round(np.std(results_svm) * 100, decimals=3)
print("SVM has", acc_svm, "% accuracy with", err_svm, "% error\n(estimated with", len(results_svm), "fold cross validation)")

SVM has 82.359 % accuracy with 0.971 % error
(estimated with 10 fold cross validation)


#### k Nearest Neighbours

In [50]:
clf_knn = KNeighborsClassifier(n_neighbors=5)
results_knn = cross_val_score(clf_knn, data, y, cv=len(results_major))

In [55]:
acc_knn = np.round(np.mean(results_knn) * 100, decimals=3)
err_knn = np.round(np.std(results_knn) * 100, decimals=3)
print("kNN has", acc_knn, "% accuracy with", err_knn, "% error\n(estimated with", len(results_knn), "fold cross validation)")

kNN has 40.739 % accuracy with 1.165 % error
(estimated with 10 fold cross validation)


#### Random Forest

In [57]:
clf_rf = RandomForestClassifier()
results_rf = cross_val_score(clf_rf, data, y, cv=len(results_major))

In [58]:
acc_rf = np.round(np.mean(results_rf) * 100, decimals=3)
err_rf = np.round(np.std(results_rf) * 100, decimals=3)
print("RF has", acc_rf, "% accuracy with", err_rf, "% error\n(estimated with", len(results_rf), "fold cross validation)")

RF has 76.311 % accuracy with 0.702 % error
(estimated with 10 fold cross validation)


#### Neural Network

In [104]:
# IMPORTS - PART 2
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import normalize, relu

from constants import path_to_bin, path_to_pssmP, path_ID, aa_order
from feature_extraction import append_truths

In [134]:
# DEFINE DATASET - consists of binary representation of AA sequence (N x 20)
#                  and PSSM matrix (N x 20)
class Enzymes(Dataset):
    def __init__(self, bin_path, pssm_path, data, names, transform):
        super().__init__()
        self.bin_path = bin_path
        self.pssm_path = pssm_path
        self.data = data
        self.names = self.data.index
        self.transform = transform
        
    def __len__(self):
        return len(self.names)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        bin_name = os.path.join(self.bin_path, names[idx])
        pssm_name = os.path.join(self.pssm_path, names[idx])
        bin_im = np.loadtxt(bin_name)
        pssm_im = np.loadtxt(pssm_name)
        feat = self.data.loc[names[idx].split('.')[0], :].to_numpy()
        sample = {'bin': bin_im, 'pssm': pssm_im, 'feat':feat}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
        

In [135]:
# DEFINE TRANSFORMATION - such that binary and PSSM will act as color channels
class ToTensors():
    
    def shuffle(self, x):
        return x[:, aa_order]
    
    def __call__(self, sample):
        bin_im, pssm_im = sample['bin'], sample['pssm']
        bin_im = self.shuffle(bin_im).transpose((1, 0))
        pssm_im = self.shuffle(pssm_im).transpose((1, 0))
        return {'bin': torch.tensor([bin_im], dtype=torch.float32),
               'pssm': torch.tensor([pssm_im], dtype=torch.float32),
               'feat': torch.tensor(sample['feat'], dtype=torch.float32)}
        
        return {'bin': normalize(torch.tensor([bin_im], dtype=torch.float32)),
               'pssm': normalize(torch.tensor([pssm_im], dtype=torch.float32))}
        

In [123]:
# READ FILE WITH FILE NAMES (enzyme ID codes)
with open(path_ID) as handle:
    names = [name.strip() for name in handle.readlines() if name.split('.')[0] in data.index]

In [136]:
# CREATE DATASET
data_cnn = Enzymes(path_to_bin, path_to_pssmP, data, names, transform=ToTensors())
data_cnn = DataLoader(data_cnn, num_workers=4)

# GROUND TRUTH
labels = pd.get_dummies(append_truths(pd.DataFrame(index=[name.split('.')[0] for name in names]))).to_numpy()

folds = KFold(n_splits=10)

In [148]:
# DEFINE NEURAL NETWORK
class Neural(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 2)
        self.conv2 = nn.Conv2d(32, 64, 2)
        self.conv3 = nn.Conv2d(1, 32, 2)
        self.conv4 = nn.Conv2d(32, 64, 2)
        
        self.pool = nn.MaxPool2d(3, 2)
        self.full_con1 = nn.Linear(1536, 256)
        self.full_con2 = nn.Linear(1536, 256)
        self.full_con3 = nn.Linear(763, 256)
        self.full_con4 = nn.Linear(256, 128)
        self.full_con5 = nn.Linear(256, 128)
        self.full_con6 = nn.Linear(256, 128)
        self.full_last = nn.Linear(384, 7)
        
    def extract_bin(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        
        x, h = nn.LSTM(x.size()[-1], 8, 2)(x)
        x = torch.flatten(x, 0)
        x = relu(self.full_con1(x))
        x = relu(self.full_con4(x))
        return x
    
    def extract_pssm(self, x):
        x = self.conv3(x)
        x = self.pool(x)
        x = self.conv4(x)
        x = self.pool(x)
        
        x, h = nn.LSTM(x.size()[-1], 8, 2)(x)
        x = torch.flatten(x, 0)
        x = relu(self.full_con2(x))
        x = relu(self.full_con5(x))
        return x
        
    def forward(self, x):
        x1 = self.extract_bin(x['bin'])
        x2 = self.extract_pssm(x['pssm'])
        x3 = self.full_con3(x['feat'])
        x3 = relu(self.full_con6(x3))
        x = torch.cat((x1, x2, x3))
        x = self.full_last(x)
        return x
net = Neural()

In [138]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=0.002)

In [149]:
for epoch in range(1):
    running_loss = 0.0
    for i, data in enumerate(zip(data_cnn.dataset, labels)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, label = data

        # forward + backward + optimize
        net.zero_grad(set_to_none=True)
        outputs = net(inputs)
        loss = criterion(outputs, torch.tensor(label, dtype=torch.float32))
        loss.backward()
        optimizer.step()
        if i > 18000:
            break

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.910
[1,  4000] loss: 1.918
[1,  6000] loss: 1.909
[1,  8000] loss: 1.907
[1, 10000] loss: 1.905
[1, 12000] loss: 1.917
[1, 14000] loss: 1.922
[1, 16000] loss: 1.921
[1, 18000] loss: 1.921
Finished Training


In [85]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

In [86]:
torch.save(net.state_dict(), './model_cnn.pth')

In [150]:
res = []
for i, x in enumerate(data_cnn.dataset):
    if i < 18001:
        continue
    res.append(np.argmax(softmax(net(x).detach().numpy()))+1)

In [151]:
true = np.argmax(labels[18001:], axis=1) + 1
b = pd.Series(true, index=names[18001:]) == pd.Series(res, index=names[18001:])
print(np.round(b.mean()*100, decimals=3))

24.02
